In [119]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/submission_format.csv
/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/Task 1/dev.csv
/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/Task 1/train.csv
/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/Task 1/test.csv
/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/Task 2/dev.csv
/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/Task 2/train.csv
/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/Task 2/test.csv


In [120]:
import warnings
warnings.filterwarnings("ignore")

In [121]:
df_train = pd.read_csv("/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/Task 1/train.csv")

In [122]:
df_train.head()

,tweet_text,claim
0,Coronavirus may have originated in lab linked ...,1
1,@SCMPNews China will buy all shares that's y i...,1
2,I'm curious if the pneumonia vaccine (Prevnar-...,1
3,dear you knew about covid 19 in january it is ...,1
4,"Wow, they're as dumb as @realDonaldTrump sugge...",1


In [123]:
df_train.describe()

,claim
count,6986.000000
mean,0.874034
std,0.331835
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


We observe that the data is skewed towards claim=1. So, we first sample a balanced data.

In [124]:
# Separate the data into positive and negative examples based on the target variable
positive_examples = df_train[df_train['claim'] == 1]
negative_examples = df_train[df_train['claim'] == 0]

min_count = min(len(positive_examples), len(negative_examples))

# Sample 50% of the positive examples
positive_sampled = positive_examples.sample(n=min_count, random_state=42)

# Sample 50% of the negative examples
negative_sampled = negative_examples.sample(n=min_count, random_state=42)

# Concatenate the sampled data back together
balanced_df_train = pd.concat([positive_sampled, negative_sampled])

# Shuffle the DataFrame to randomize the order of samples
balanced_df_train = balanced_df_train.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the distribution of classes in the balanced dataset
print(balanced_df_train['claim'].value_counts())

claim
0    880
1    880
Name: count, dtype: int64


In [125]:
df_train = balanced_df_train

### Preprocessing

In [126]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

# Download NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [127]:
!python3 -m nltk.downloader wordnet
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

/opt/conda/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
replace /usr/share/nltk_data/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [128]:
# Define preprocessing function
def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Convert text to lowercase
    text = text.lower()
    
    # Tokenize text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join tokens back into text
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

# Apply preprocessing to tweet_text column
df_train['preprocessed_text'] = df_train['tweet_text'].apply(preprocess_text)

In [129]:
df_train.head()

,tweet_text,claim,preprocessed_text
0,So here’s a question for today: if you are a d...,0,’ question today doctor seattle area today ord...
1,Wish he becomes the role model of getting cure...,0,wish becomes role model getting cured self iso...
2,Copper Kills Coronavirus. Why Aren’t Our Surfa...,0,copper kill coronavirus ’ surface covered http...
3,Let’s hope #Covid_19 isn’t coming back once a ...,0,let ’ hope covid19 ’ coming back cure vaccine ...
4,#NowThis\n...people need to be reminded that #...,1,nowthis people need reminded coronavirus test ...


In [130]:
# Split data into features and labels
X = df_train['preprocessed_text']
y = df_train['claim']

In [131]:
X_train = X
y_train = y

In [70]:
# loading the validation data for future use
df_val = pd.read_csv("/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/Task 1/dev.csv")
df_val.head()

,tweet_text,claim
0,friends the stuff about nature healing and wil...,1
1,A9. If you’re thinking of taking action based ...,1
2,"Kyle Turley, co-owner of Moreno Valley dispens...",1
3,exhibit a,0
4,The English edition of Marathi daily Lokmat pu...,1


In [71]:
df_val.describe()

,claim
count,1497.000000
mean,0.869071
std,0.337435
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [72]:
#preprocess
df_val['preprocessed_text'] = df_val['tweet_text'].apply(preprocess_text)
# Split data into features and labels

X_val = df_val['preprocessed_text']
y_val = df_val['claim']

# Calculate lengths of each sentence in X_train
sentence_lengths = [len(sentence.split()) for sentence in X_val]

# Calculate mean and maximum length
mean_length = sum(sentence_lengths) / len(sentence_lengths)
max_length = max(sentence_lengths)

print("Mean length of sentences:", mean_length)
print("Maximum length of sentences:", max_length)

Mean length of sentences: 16.733466933867735
Maximum length of sentences: 50


### Word Embedding

In [135]:
!pip install tensorflow

In [136]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Initialize Tokenizer
tokenizer = Tokenizer()

# Fit tokenizer on training data
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_sequences = tokenizer.texts_to_sequences(X_train)

# Pad sequences to ensure uniform length
max_seq_length = 90  # adjust as needed
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_seq_length)

# Print vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary size:", vocab_size)

Vocabulary size: 7909


In [137]:
#exploring the data

#get mean and max length of sentences in X_train
# Calculate lengths of each sentence in X_train
sentence_lengths = [len(sentence.split()) for sentence in X_train]

# Calculate mean and maximum length
mean_length = sum(sentence_lengths) / len(sentence_lengths)
max_length = max(sentence_lengths)

print("Mean length of sentences:", mean_length)
print("Maximum length of sentences:", max_length)

Mean length of sentences: 15.591477272727273
Maximum length of sentences: 46


### Model : The RNN

In [18]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense

# Define RNN model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100))
model.add(SimpleRNN(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
print(model.summary())

# Train the model
history = model.fit(X_train_padded, y_train, batch_size=32, epochs=10, validation_split=0.2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         2251600   
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                10560     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2262225 (8.63 MB)
Trainable params: 2262225 (8.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10


I0000 00:00:1710616710.512447     124 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


175/175 [==============================] - 34s 177ms/step - loss: 0.4023 - accuracy: 0.8663 - val_loss: 0.3450 - val_accuracy: 0.8763
Epoch 2/10
175/175 [==============================] - 22s 123ms/step - loss: 0.2365 - accuracy: 0.8898 - val_loss: 0.3733 - val_accuracy: 0.8805
Epoch 3/10
175/175 [==============================] - 17s 96ms/step - loss: 0.1065 - accuracy: 0.9617 - val_loss: 0.4587 - val_accuracy: 0.8526
Epoch 4/10
175/175 [==============================] - 17s 99ms/step - loss: 0.0616 - accuracy: 0.9810 - val_loss: 0.5629 - val_accuracy: 0.8205
Epoch 5/10
175/175 [==============================] - 16s 91ms/step - loss: 0.0312 - accuracy: 0.9902 - val_loss: 0.5577 - val_accuracy: 0.8469
Epoch 6/10
175/175 [==============================] - 15s 87ms/step - loss: 0.0190 - accuracy: 0.9925 - val_loss: 0.7383 - val_accuracy: 0.8398
Epoch 7/10
175/175 [==============================] - 16s 90ms/step - loss: 0.0162 - accuracy: 0.9927 - val_loss: 0.6981 - val_accuracy: 0.8469
E

In [28]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 90, 100)        │     2,251,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 64)             │        10,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,786,677 (25.89 MB)

 Trainable params: 2,262,225 (8.63 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,524,452 (17.26 MB)

In [32]:
# Evaluate the model on training data
train_loss, train_accuracy = model.evaluate(X_train_padded, y_train)
print("Training Loss:", train_loss)
print("Training Accuracy:", train_accuracy)

# Evaluate the model on validation data
X_val_sequences = tokenizer.texts_to_sequences(X_val)  # Tokenize validation data
X_val_padded = pad_sequences(X_val_sequences, maxlen=max_seq_length)  # Pad validation sequences
val_loss, val_accuracy = model.evaluate(X_val_padded, y_val)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9914 - loss: 0.0265
Training Loss: 0.16137950122356415
Training Accuracy: 0.9655024409294128
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8282 - loss: 0.9019
Validation Loss: 0.8503451943397522
Validation Accuracy: 0.8336673378944397


In [19]:
rnn_model = model #saved the rnn model

#### Model Part 1.5 : The RNN saved earlier

In [74]:
from keras.callbacks import ModelCheckpoint

from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense

# Define RNN model with mask_zero and saving history
model_rnn_best = Sequential()
model_rnn_best.add(Embedding(input_dim=vocab_size, output_dim=100, mask_zero=True))
model_rnn_best.add(SimpleRNN(units=64, activation='relu'))
model_rnn_best.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model_rnn_best.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define checkpoint callback
checkpoint = ModelCheckpoint("best_rnn_val.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Train the model with checkpoint callback
history_rnn_best = model_rnn_best.fit(X_train_padded, y_train, batch_size=32, epochs=10, validation_split=0.2, callbacks=[checkpoint])

Epoch 1/10
44/44 [==============================] - ETA: 0s - loss: 0.6868 - accuracy: 0.5327
Epoch 1: val_accuracy improved from -inf to 0.56818, saving model to best_rnn_val.h5
44/44 [==============================] - 9s 173ms/step - loss: 0.6868 - accuracy: 0.5327 - val_loss: 0.6837 - val_accuracy: 0.5682
Epoch 2/10
44/44 [==============================] - ETA: 0s - loss: 0.5645 - accuracy: 0.8352
Epoch 2: val_accuracy improved from 0.56818 to 0.58807, saving model to best_rnn_val.h5
44/44 [==============================] - 6s 140ms/step - loss: 0.5645 - accuracy: 0.8352 - val_loss: 0.6569 - val_accuracy: 0.5881
Epoch 3/10
44/44 [==============================] - ETA: 0s - loss: 0.2063 - accuracy: 0.9538
Epoch 3: val_accuracy improved from 0.58807 to 0.59375, saving model to best_rnn_val.h5
44/44 [==============================] - 5s 114ms/step - loss: 0.2063 - accuracy: 0.9538 - val_loss: 0.7700 - val_accuracy: 0.5938
Epoch 4/10
44/44 [==============================] - ETA: 0s - lo

In [138]:
from keras.models import load_model

In [77]:
# Evaluate the model on training data
# Load the saved model
loaded_model = load_model("best_rnn_val.h5")
train_loss, train_accuracy = loaded_model.evaluate(X_train_padded, y_train)
print("Training Loss:", train_loss)
print("Training Accuracy:", train_accuracy)

# Evaluate the model on validation data
X_val_sequences = tokenizer.texts_to_sequences(X_val)  # Tokenize validation data
max_val_length = max(len(sequence) for sequence in X_val_sequences)
X_val_padded = pad_sequences(X_val_sequences, maxlen=max_val_length)  # Pad validation sequences
val_loss, val_accuracy = loaded_model.evaluate(X_val_padded, y_val)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

55/55 [==============================] - 1s 12ms/step - loss: 0.2040 - accuracy: 0.9187
Training Loss: 0.20403744280338287
Training Accuracy: 0.918749988079071
47/47 [==============================] - 1s 7ms/step - loss: 0.9838 - accuracy: 0.5979
Validation Loss: 0.9838300347328186
Validation Accuracy: 0.5978623628616333


### Model Part II: The LSTM (with dropout)

In [20]:
#lets create LSTM model with dropouts
from keras.layers import LSTM, Dropout

# Define enhanced model with LSTM and Dropout
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=vocab_size, output_dim=100))
lstm_model.add(LSTM(units=64, activation='tanh', return_sequences=True))  # LSTM layer with return_sequences=True for stacking
lstm_model.add(Dropout(0.2))  # Dropout layer to reduce overfitting
lstm_model.add(LSTM(units=32, activation='tanh'))  # Second LSTM layer
lstm_model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history_lstm = lstm_model.fit(X_train_padded, y_train, batch_size=32, epochs=10, validation_split=0.2)

Epoch 1/10
175/175 [==============================] - 19s 78ms/step - loss: 0.3866 - accuracy: 0.8724 - val_loss: 0.3545 - val_accuracy: 0.8763
Epoch 2/10
175/175 [==============================] - 6s 34ms/step - loss: 0.2780 - accuracy: 0.8885 - val_loss: 0.3548 - val_accuracy: 0.8684
Epoch 3/10
175/175 [==============================] - 4s 22ms/step - loss: 0.1412 - accuracy: 0.9488 - val_loss: 0.4382 - val_accuracy: 0.8698
Epoch 4/10
175/175 [==============================] - 3s 16ms/step - loss: 0.0791 - accuracy: 0.9723 - val_loss: 0.5524 - val_accuracy: 0.8433
Epoch 5/10
175/175 [==============================] - 3s 16ms/step - loss: 0.0527 - accuracy: 0.9837 - val_loss: 0.5233 - val_accuracy: 0.8097
Epoch 6/10
175/175 [==============================] - 3s 17ms/step - loss: 0.0378 - accuracy: 0.9884 - val_loss: 0.6572 - val_accuracy: 0.8355
Epoch 7/10
175/175 [==============================] - 3s 15ms/step - loss: 0.0267 - accuracy: 0.9905 - val_loss: 0.6795 - val_accuracy: 0.856

In [21]:
#testing the LSTM
# Evaluate the model on training data
train_loss, train_accuracy = lstm_model.evaluate(X_train_padded, y_train)
print("Training Loss:", train_loss)
print("Training Accuracy:", train_accuracy)

# Evaluate the model on validation data
X_val_sequences = tokenizer.texts_to_sequences(X_val)  # Tokenize validation data
X_val_padded = pad_sequences(X_val_sequences, maxlen=max_seq_length)  # Pad validation sequences
val_loss, val_accuracy = lstm_model.evaluate(X_val_padded, y_val)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

219/219 [==============================] - 1s 6ms/step - loss: 0.1670 - accuracy: 0.9642
Training Loss: 0.16704432666301727
Training Accuracy: 0.9642141461372375
47/47 [==============================] - 0s 5ms/step - loss: 0.9125 - accuracy: 0.8410
Validation Loss: 0.9125417470932007
Validation Accuracy: 0.8410153388977051


### Model Part 2.5: The LSTM with no padding

In [139]:
#lets create LSTM model with dropouts
from keras.layers import LSTM, Dropout

In [23]:
# Define enhanced model with LSTM and Dropout
lstm_model_pad = Sequential()
lstm_model_pad.add(Embedding(input_dim=vocab_size, output_dim=100, mask_zero = True))
lstm_model_pad.add(LSTM(units=64, activation='tanh', return_sequences=True))  # LSTM layer with return_sequences=True for stacking
lstm_model_pad.add(Dropout(0.2))  # Dropout layer to reduce overfitting
lstm_model_pad.add(LSTM(units=32, activation='tanh'))  # Second LSTM layer
lstm_model_pad.add(Dense(units=1, activation='sigmoid'))

# Compile the model
lstm_model_pad.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history_lstm_pad = lstm_model_pad.fit(X_train_padded, y_train, batch_size=32, epochs=10, validation_split=0.2)

Epoch 1/10
175/175 [==============================] - 89s 458ms/step - loss: 0.3755 - accuracy: 0.8713 - val_loss: 0.3266 - val_accuracy: 0.8777
Epoch 2/10
175/175 [==============================] - 68s 390ms/step - loss: 0.2251 - accuracy: 0.9086 - val_loss: 0.3681 - val_accuracy: 0.8763
Epoch 3/10
175/175 [==============================] - 67s 385ms/step - loss: 0.1163 - accuracy: 0.9596 - val_loss: 0.4524 - val_accuracy: 0.8591
Epoch 4/10
175/175 [==============================] - 66s 377ms/step - loss: 0.0772 - accuracy: 0.9757 - val_loss: 0.5020 - val_accuracy: 0.8176
Epoch 5/10
175/175 [==============================] - 66s 374ms/step - loss: 0.0574 - accuracy: 0.9823 - val_loss: 0.5746 - val_accuracy: 0.8205
Epoch 6/10
175/175 [==============================] - 66s 379ms/step - loss: 0.0435 - accuracy: 0.9875 - val_loss: 0.5891 - val_accuracy: 0.8391
Epoch 7/10
175/175 [==============================] - 66s 378ms/step - loss: 0.0355 - accuracy: 0.9889 - val_loss: 0.7237 - val_ac

In [24]:
#testing the LSTM (ignore pad model)
# Evaluate the model on training data
train_loss, train_accuracy = lstm_model_pad.evaluate(X_train_padded, y_train)
print("Training Loss:", train_loss)
print("Training Accuracy:", train_accuracy)

# Evaluate the model on validation data
X_val_sequences = tokenizer.texts_to_sequences(X_val)  # Tokenize validation data
X_val_padded = pad_sequences(X_val_sequences, maxlen=max_seq_length)  # Pad validation sequences
val_loss, val_accuracy = lstm_model_pad.evaluate(X_val_padded, y_val)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

219/219 [==============================] - 8s 38ms/step - loss: 0.2064 - accuracy: 0.9586
Training Loss: 0.20639245212078094
Training Accuracy: 0.9586315751075745
47/47 [==============================] - 2s 38ms/step - loss: 1.0837 - accuracy: 0.8103
Validation Loss: 1.083691120147705
Validation Accuracy: 0.8102872371673584


### Model : LSTM with best saved

In [143]:
from keras.callbacks import ModelCheckpoint

from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense

# Define enhanced model with LSTM and Dropout
lstm_model_pad_best = Sequential()
lstm_model_pad_best.add(Embedding(input_dim=vocab_size, output_dim=100, mask_zero = True))
lstm_model_pad_best.add(LSTM(units=64, activation='tanh', return_sequences=True))  # LSTM layer with return_sequences=True for stacking
lstm_model_pad_best.add(Dropout(0.2))  # Dropout layer to reduce overfitting
lstm_model_pad_best.add(LSTM(units=32, activation='tanh'))  # Second LSTM layer
lstm_model_pad_best.add(Dense(units=1, activation='sigmoid'))

# Compile the model
lstm_model_pad_best.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define checkpoint callback
checkpoint = ModelCheckpoint("best_lstm_val.keras", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Train the model with checkpoint callback
history_lstm_best = lstm_model_pad_best.fit(X_train_padded, y_train, batch_size=32, epochs=10, validation_split=0.2, callbacks=[checkpoint])

Epoch 1/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - accuracy: 0.4867 - loss: 0.6886
Epoch 1: val_accuracy improved from -inf to 0.67330, saving model to best_lstm_val.keras
44/44 ━━━━━━━━━━━━━━━━━━━━ 12s 197ms/step - accuracy: 0.4879 - loss: 0.6883 - val_accuracy: 0.6733 - val_loss: 0.6252
Epoch 2/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.8451 - loss: 0.4308
Epoch 2: val_accuracy did not improve from 0.67330
44/44 ━━━━━━━━━━━━━━━━━━━━ 8s 189ms/step - accuracy: 0.8449 - loss: 0.4301 - val_accuracy: 0.6506 - val_loss: 0.7030
Epoch 3/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - accuracy: 0.9467 - loss: 0.1684
Epoch 3: val_accuracy did not improve from 0.67330
44/44 ━━━━━━━━━━━━━━━━━━━━ 8s 192ms/step - accuracy: 0.9467 - loss: 0.1683 - val_accuracy: 0.6364 - val_loss: 0.9032
Epoch 4/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.9788 - loss: 0.0740
Epoch 4: val_accuracy did not improve from 0.67330
44/44 ━━━━━━━━━━━━━━━━━━━━ 8s 191ms/step - accuracy: 0.9788 - 

In [141]:
# Evaluate the model on training data
# Load the saved model
best_lstm_model = load_model("best_lstm_val.keras")
train_loss, train_accuracy = best_lstm_model.evaluate(X_train_padded, y_train)
print("Training Loss:", train_loss)
print("Training Accuracy:", train_accuracy)

# Evaluate the model on validation data

X_val_sequences = tokenizer.texts_to_sequences(X_val)  # Tokenize validation data
max_val_length = max(len(sequence) for sequence in X_val_sequences)
X_val_padded = pad_sequences(X_val_sequences, maxlen=max_val_length)  # Pad validation sequences
val_loss, val_accuracy = best_lstm_model.evaluate(X_val_padded, y_val)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - accuracy: 0.9842 - loss: 0.0709
Training Loss: 0.27611351013183594
Training Accuracy: 0.9227272868156433


NameError: name 'X_val' is not defined

### LSTM with dropout, regularisation, normaliastion

In [98]:
from keras.layers import Dropout, BatchNormalization, LSTM
from keras.regularizers import l2

# Define enhanced model with LSTM, Dropout, Regularization, and Batch Normalization
lstm_model_pad_norm_best = Sequential()
lstm_model_pad_norm_best.add(Embedding(input_dim=vocab_size, output_dim=100, mask_zero=True))
lstm_model_pad_norm_best.add(LSTM(units=64, activation='tanh', return_sequences=True, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))  # LSTM layer with return_sequences=True for stacking
lstm_model_pad_norm_best.add(Dropout(0.2))  # Dropout layer to reduce overfitting
lstm_model_pad_norm_best.add(BatchNormalization())  # Batch normalization layer
lstm_model_pad_norm_best.add(LSTM(units=32, activation='tanh', kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))  # Second LSTM layer
lstm_model_pad_norm_best.add(Dropout(0.2))  # Dropout layer to reduce overfitting
lstm_model_pad_norm_best.add(BatchNormalization())  # Batch normalization layer
lstm_model_pad_norm_best.add(Dense(units=1, activation='sigmoid'))

# Compile the model
lstm_model_pad_norm_best.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define checkpoint callback
checkpoint = ModelCheckpoint("best_lstm_norm_val.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Train the model with checkpoint callback
history_lstm_norm_best = lstm_model_pad_norm_best.fit(X_train_padded, y_train, batch_size=32, epochs=15, validation_split=0.2, callbacks=[checkpoint])


Epoch 1/15
44/44 [==============================] - ETA: 0s - loss: 4.0383 - accuracy: 0.6080
Epoch 1: val_accuracy improved from -inf to 0.65341, saving model to best_lstm_norm_val.h5
44/44 [==============================] - 22s 327ms/step - loss: 4.0383 - accuracy: 0.6080 - val_loss: 3.2900 - val_accuracy: 0.6534
Epoch 2/15
44/44 [==============================] - ETA: 0s - loss: 2.4881 - accuracy: 0.8750
Epoch 2: val_accuracy improved from 0.65341 to 0.65909, saving model to best_lstm_norm_val.h5
44/44 [==============================] - 10s 237ms/step - loss: 2.4881 - accuracy: 0.8750 - val_loss: 2.5134 - val_accuracy: 0.6591
Epoch 3/15
44/44 [==============================] - ETA: 0s - loss: 1.7434 - accuracy: 0.9553
Epoch 3: val_accuracy did not improve from 0.65909
44/44 [==============================] - 10s 224ms/step - loss: 1.7434 - accuracy: 0.9553 - val_loss: 2.1001 - val_accuracy: 0.6591
Epoch 4/15
44/44 [==============================] - ETA: 0s - loss: 1.3564 - accuracy:

In [142]:
# Evaluate the model on training data
# Load the saved model
best_lstm_norm_model = load_model("best_lstm_norm_val.h5")
train_loss, train_accuracy = best_lstm_norm_model.evaluate(X_train_padded, y_train)
print("Training Loss:", train_loss)
print("Training Accuracy:", train_accuracy)

# Evaluate the model on validation data

X_val_sequences = tokenizer.texts_to_sequences(X_val)  # Tokenize validation data
max_val_length = max(len(sequence) for sequence in X_val_sequences)
X_val_padded = pad_sequences(X_val_sequences, maxlen=max_val_length)  # Pad validation sequences
val_loss, val_accuracy = best_lstm_norm_model.evaluate(X_val_padded, y_val)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'best_lstm_norm_val.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## Testing

In [196]:
df_test = pd.read_csv('/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/submission_format.csv')
df_test = pd.read_csv('/kaggle/input/ai-hackthon-aravind/ans.csv')
df_test = pd.read_csv('/kaggle/input/ai-hackathon-ara-2/ans (1).csv')
df_test.head()

,Unnamed: 0,ID,text,claim,span_start_index,span_end_index,task
0,0,0,Of course we should have captured Osama Bin La...,1,[-2],[-2],1
1,1,1,covid19 will end soon amen covid19 will end so...,0,[-2],[-2],1
2,2,2,#Coronavirus #SanDiego #1 #Hotspot #92103 # Co...,1,[-2],[-2],1
3,3,3,@ICannot_Enough @elonmusk Yes it is. Because $...,0,[-2],[-2],1
4,4,4,Some people are saying black people are immune...,1,[-2],[-2],1


In [146]:
#note: 17th march 0111hrs, using GPU x2 T4

In [147]:
#preprocess
df_test['preprocessed_text'] = df_test['text'].apply(preprocess_text)
X_test = df_test['preprocessed_text']
# Split data into features and labels
# Calculate lengths of each sentence in X_train
sentence_lengths = [len(sentence.split()) for sentence in X_test]

# Calculate mean and maximum length
mean_length = sum(sentence_lengths) / len(sentence_lengths)
max_length = max(sentence_lengths)

print("Mean length of sentences:", mean_length)
print("Maximum length of sentences:", max_length)

Mean length of sentences: 16.655570350643586
Maximum length of sentences: 74


In [148]:
!pip install tqdm
from tqdm import tqdm

In [149]:
cur_model = best_lstm_model
cur_max_seq_length = max_length

X_test_sequences = tokenizer.texts_to_sequences(X_test)  # Tokenize validation data
max_test_length = max(len(sequence) for sequence in X_test_sequences)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_test_length)
predictions = cur_model.predict(X_test_padded, verbose=False)
predictions = (predictions > 0.5).astype(int)

for index, row in tqdm(df_test.iterrows(), total=len(df_test), leave=True):
    if row['task'] == 1:  # Check if task is 1
        # Update the 'claim' column with the model prediction
        df_test.at[index, 'claim'] = predictions[index]

# Save the modified DataFrame to a new CSV file
df_test.drop(columns=['preprocessed_text'], inplace=True)
df_test.to_csv('submission.csv', index=False)

100%|██████████| 2253/2253 [00:00<00:00, 12067.64it/s]


In [39]:
df_test.iloc[[224]]

,tweet_text,preprocessed_text
224,Congratulations 🥳🥳🥳Since our police force have...,congratulation 🥳🥳🥳since police force finally f...


# Task 2 : Claim Span Prediction

In [150]:
df_train2 = pd.read_csv("/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/Task 2/train.csv")

In [151]:
df_train2.head()

,tokens,span_start_index,span_end_index
0,"['""who', ' may', ' (or', ' may', ' not', ') ha...",[43],[53]
1,"['RT', ' @Coach_Brod', ': If', ' you', ' have'...",[2],[17]
2,"['#Pharmacists', ' warn', ' against', ' #malar...",[0],[4]
3,"['You', ' got', ' to', ' boil', ' your', ' Clo...","[0, 22]","[20, 33]"
4,"['There', ' is', ' no', ' virus', '. \nAnd', '...",[0],[3]


In [152]:
from nltk.stem import WordNetLemmatizer
import string

# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Define your function to process a word
def process_word(word):
    # Remove punctuation
    word = word.translate(str.maketrans('', '', string.punctuation))
    # Lemmatize the word
    word = lemmatizer.lemmatize(word)
    return word

#convert strings to lists
def process_row(row):
    row['tokens'] = eval(row['tokens'])
    row['span_start_index'] = eval(row['span_start_index'])
    row['span_end_index'] = eval(row['span_end_index'])
    row['tokens'] = [process_word(word) for word in row['tokens']]
    return row

df_train2 = df_train2.apply(process_row, axis = 1)

In [153]:
df_train2.head()

,tokens,span_start_index,span_end_index
0,"[who, may, or, may, not, have, it, Sch...",[43],[53]
1,"[RT, CoachBrod, If, you, have, ever, suc...",[2],[17]
2,"[Pharmacists, warn, against, malarial, dru...",[0],[4]
3,"[You, got, to, boil, your, Clorox, to, ...","[0, 22]","[20, 33]"
4,"[There, is, no, virus, \nAnd, if, there,...",[0],[3]


In [154]:
df_train2.describe()

,tokens,span_start_index,span_end_index
count,6044,6044,6044
unique,6029,688,833
top,"[Lysol, and, Dettol, manufacturer, tells, ...",[0],[9]
freq,3,1455,288


In [155]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

X_train = df_train2['tokens']

# Calculate vocabulary size
vocab = set()
for tokens in X_train:
    vocab.update(tokens)
vocab_size = len(vocab)
print("Vocabulary size:", vocab_size)

# Define a function to generate binary labels for each word
def generate_binary_labels(tokens, span_start_index, span_end_index):
    if not span_start_index:  # Check if span_start_index is empty
        return [0] * len(tokens)  # Return a list of zeros if it is empty
    binary_labels = []
    for i, token in enumerate(tokens):
        if i in range(span_start_index[0], span_end_index[0] + 1):
            binary_labels.append(1)  # Word is part of the segment
        else:
            binary_labels.append(0)  # Word is not part of the segment
    return binary_labels

# Generate binary labels for each sample
y_train = [generate_binary_labels(tokens, start_idx, end_idx) for tokens, start_idx, end_idx in zip(df_train2['tokens'], df_train2['span_start_index'], df_train2['span_end_index'])]

Vocabulary size: 26950


In [156]:
max_seq_len = max(len(tokens) for tokens in df_train2['tokens'])

# Generate binary labels for each sample and pad sequences
y_train = []
for tokens, start_idx, end_idx in zip(df_train2['tokens'], df_train2['span_start_index'], df_train2['span_end_index']):
    binary_labels = generate_binary_labels(tokens, start_idx, end_idx)
    # Pad sequences if necessary
    if len(binary_labels) < max_seq_len:
        binary_labels += [0] * (max_seq_len - len(binary_labels))
    elif len(binary_labels) > max_seq_len:
        binary_labels = binary_labels[:max_seq_len]
    y_train.append(binary_labels)

y_train = np.array(y_train)

In [157]:
# Convert set vocab to list and sort it
vocab_list = sorted(list(vocab))

# Create a dictionary to map tokens to indices
token_to_index = {token: index for index, token in enumerate(vocab_list)}

# Calculate vocabulary size
vocab_size = len(vocab_list)

In [158]:
from keras.preprocessing.sequence import pad_sequences

# Convert tokens to sequences
X_train_sequences = [[token_to_index[token] for token in tokens] for tokens in df_train2['tokens']]

# Pad sequences
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_seq_len, padding='post', truncating='post')


In [159]:
type(X_train_padded)

numpy.ndarray

In [160]:
# Define the model architecture
embedding_dim = 128  # Adjust as needed
lstm_units = 64  # Adjust as needed

lstm_model2 = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),  # None allows variable sequence lengths
    LSTM(units=lstm_units, activation='tanh', return_sequences=True),
    Dense(units=1, activation='sigmoid')
])

# Compile the model
lstm_model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
lstm_model2.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8553 - loss: 0.2442 - val_accuracy: 0.9118 - val_loss: 0.1700
Epoch 2/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9279 - loss: 0.1508 - val_accuracy: 0.9152 - val_loss: 0.1678
Epoch 3/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9527 - loss: 0.1083 - val_accuracy: 0.9125 - val_loss: 0.1844
Epoch 4/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9684 - loss: 0.0773 - val_accuracy: 0.9125 - val_loss: 0.2153
Epoch 5/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9747 - loss: 0.0624 - val_accuracy: 0.9135 - val_loss: 0.2315
Epoch 6/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9797 - loss: 0.0512 - val_accuracy: 0.9113 - val_loss: 0.2663
Epoch 7/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9830 - loss: 0.0433 - val_accuracy: 0.9125 - val_loss: 0.2929
Epoch 8/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9860 - loss: 0.0358 - val_accuracy: 0

In [207]:
import gensim.downloader as api

# Download the pre-trained GloVe word vectors
glove_vectors = api.load("glove-wiki-gigaword-100")

In [211]:
from keras.initializers import Constant

# Initialize an embedding matrix with zeros
embedding_matrix = np.zeros((vocab_size, 100))

# Fill the embedding matrix with word vectors from GloVe
for word, i in tokenizer.word_index.items():
    if word in glove_vectors:
        embedding_matrix[i] = glove_vectors[word]

# Define the model architecture with pre-trained GloVe embeddings
lstm_model_with_glove = Sequential([
    Embedding(input_dim=vocab_size, output_dim=100, embeddings_initializer=Constant(embedding_matrix), trainable=False),  
    LSTM(units=lstm_units, activation='tanh', return_sequences=True),
    Dense(units=1, activation='sigmoid')
])

# Compile the model
lstm_model_with_glove.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
lstm_model_with_glove.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8764 - loss: 0.2436 - val_accuracy: 0.9048 - val_loss: 0.1758
Epoch 2/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9226 - loss: 0.1560 - val_accuracy: 0.9123 - val_loss: 0.1700
Epoch 3/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9452 - loss: 0.1199 - val_accuracy: 0.9109 - val_loss: 0.1819
Epoch 4/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9627 - loss: 0.0886 - val_accuracy: 0.9108 - val_loss: 0.2052
Epoch 5/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9719 - loss: 0.0683 - val_accuracy: 0.9121 - val_loss: 0.2355
Epoch 6/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9783 - loss: 0.0544 - val_accuracy: 0.9120 - val_loss: 0.2666
Epoch 7/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9824 - loss: 0.0443 - val_accuracy: 0.9119 - val_loss: 0.3008
Epoch 8/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9855 - loss: 0.0377 - val_accuracy: 0

In [233]:
from keras.layers import Dropout, Bidirectional, GlobalMaxPooling1D

In [235]:
# Define the model architecture with additional layers
lstm_model_with_dropout = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),  
    LSTM(units=lstm_units, activation='tanh', return_sequences=True),
    Dropout(0.2),  # Dropout layer to prevent overfitting
    Bidirectional(LSTM(units=lstm_units, activation='tanh', return_sequences=True)),  # Bidirectional LSTM layer for better context understanding
    Dropout(0.2),  # Dropout layer
    Dense(units=64, activation='relu'),  # Additional Dense layer for more complex mappings
    Dropout(0.2),  # Dropout layer
    Dense(units=1, activation='sigmoid')
])

# Compile the model
lstm_model_with_dropout.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
lstm_model_with_dropout.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
149/152 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8573 - loss: 0.2650

KeyboardInterrupt: 

In [245]:
from keras.callbacks import ModelCheckpoint

# Define the model architecture with regularization
lstm_model_with_regularization = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    Bidirectional(LSTM(units=lstm_units, activation='tanh', return_sequences=True)),
    Dense(units=64, activation='relu', kernel_regularizer=l2(0.01)),  # Adding L2 regularization
    Dropout(0.5),  # Dropout for regularization
    Dense(units=32, activation='relu', kernel_regularizer=l2(0.01)),  # Adding L2 regularization
    Dropout(0.5),  # Dropout for regularization
    Dense(units=1, activation='sigmoid')
])

# Compile the model
lstm_model_with_regularization.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define checkpoint callback to save the best model
checkpoint = ModelCheckpoint("best_lstm_model_with_regularization.keras", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Train the model with checkpoint callback
history = lstm_model_with_regularization.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[checkpoint])


Epoch 1/10
151/152 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8448 - loss: 1.1311
Epoch 1: val_accuracy improved from -inf to 0.90880, saving model to best_lstm_model_with_regularization.keras
152/152 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.8452 - loss: 1.1261 - val_accuracy: 0.9088 - val_loss: 0.3089
Epoch 2/10
151/152 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9154 - loss: 0.2593
Epoch 2: val_accuracy improved from 0.90880 to 0.92040, saving model to best_lstm_model_with_regularization.keras
152/152 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9154 - loss: 0.2589 - val_accuracy: 0.9204 - val_loss: 0.2108
Epoch 3/10
151/152 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9471 - loss: 0.1522
Epoch 3: val_accuracy improved from 0.92040 to 0.92300, saving model to best_lstm_model_with_regularization.keras
152/152 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9471 - loss: 0.1522 - val_accuracy: 0.9230 - val_loss: 0.1845
Epoch 4/10
149/152 ━━━━━━━━━━━━━━━━━━━━ 0s 11m

In [249]:
from keras.models import load_model

# Load the best model
lstm_model_with_regularization.save("best_lstm_model_with_regularization.keras")


In [161]:
df_val2 = pd.read_csv('/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/Task 2/dev.csv')
df_val2.head()

,tokens,span_start_index,span_end_index
0,"['attempt', ' by', ' to', ' minimize', ' coron...",[11],[23]
1,"['I', ' do', ' not', ' defend', ' Trump', ' mu...",[15],[25]
2,"['truly', ' sobering', ' analysis', ' us', ' m...",[3],[10]
3,"['Bombshell', ': Chinese', ' Doctor', ' Experi...",[1],[12]
4,"['Now', ' I', ' get', ' it', ' from', ' #Fauci...","[7, 10]","[9, 20]"


In [162]:
df_val2 = df_val2.apply(process_row, axis = 1)
df_val2.head()

,tokens,span_start_index,span_end_index
0,"[attempt, by, to, minimize, coronavirus, ...",[11],[23]
1,"[I, do, not, defend, Trump, much, but, ...",[15],[25]
2,"[truly, sobering, analysis, us, more, vul...",[3],[10]
3,"[Bombshell, Chinese, Doctor, Experimented, ...",[1],[12]
4,"[Now, I, get, it, from, Fauci, COVID19, ...","[7, 10]","[9, 20]"


In [163]:
UNK_TOKEN_INDEX = len(vocab_list)  # Assuming it's the next index after the vocabulary size

# Convert tokens to sequences, replacing out-of-vocabulary tokens with UNK_TOKEN_INDEX
X_val_sequences = [[token_to_index.get(token, UNK_TOKEN_INDEX) for token in tokens] for tokens in df_val2['tokens']]

In [164]:
max_val_len = max(len(sequence) for sequence in X_val_sequences)
X_val_padded = pad_sequences(X_val_sequences, maxlen=max_val_len, padding='post', truncating='post')

In [165]:
y_val = [generate_binary_labels(tokens, start_idx, end_idx) for tokens, start_idx, end_idx in zip(df_val2['tokens'], df_val2['span_start_index'], df_val2['span_end_index'])]

# Generate binary labels for each sample and pad sequences
y_val = []
for tokens, start_idx, end_idx in zip(df_val2['tokens'], df_val2['span_start_index'], df_val2['span_end_index']):
    binary_labels = generate_binary_labels(tokens, start_idx, end_idx)
    # Pad sequences if necessary
    if len(binary_labels) < max_val_len:
        binary_labels += [0] * (max_val_len - len(binary_labels))
    elif len(binary_labels) > max_val_len:
        binary_labels = binary_labels[:max_val_len]
    y_val.append(binary_labels)

y_val = np.array(y_val)

In [166]:
X_val_padded.shape

(756, 59)

In [167]:
y_val.shape

(756, 59)

In [230]:
# Evaluate the model on validation data
#val_loss, val_accuracy = lstm_model2.evaluate(X_val_padded, y_val)
val_loss, val_accuracy = lstm_model_with_dropout.evaluate(X_val_padded, y_val)
# Print validation loss and accuracy
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8822 - loss: 0.5125
Validation Loss: 0.5358427166938782
Validation Accuracy: 0.8800331354141235


### Testing : Task 2

In [215]:
df_test.head()

,Unnamed: 0,ID,text,claim,span_start_index,span_end_index,task
0,0,0,Of course we should have captured Osama Bin La...,1,[-2],[-2],1
1,1,1,covid19 will end soon amen covid19 will end so...,0,[-2],[-2],1
2,2,2,#Coronavirus #SanDiego #1 #Hotspot #92103 # Co...,1,[-2],[-2],1
3,3,3,@ICannot_Enough @elonmusk Yes it is. Because $...,0,[-2],[-2],1
4,4,4,Some people are saying black people are immune...,1,[-2],[-2],1


In [216]:
df_test2 = df_test[df_test['task'] == 2]

X_test_sequences2 = [[token_to_index.get(token, UNK_TOKEN_INDEX) for token in tokens] for tokens in df_test2['text']]
max_test_len = max(len(sequence) for sequence in X_test_sequences2)
# Pad sequences
X_test_padded2 = pad_sequences(X_test_sequences2, maxlen=max_test_len, padding='post', truncating='post')

X_test_padded2 = pad_sequences(X_test_sequences2, maxlen=max_test_len, padding='post', truncating='post')

In [250]:
# Use the model to predict on the test data
#predictions = lstm_model2.predict(X_test_padded2)
#predictions = lstm_model_with_glove.predict(X_test_padded2)
predictions = lstm_model_with_regularization.predict(X_test_padded2)

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step


In [251]:
# Convert predicted probabilities to binary values
binary_predictions = (predictions > 0.5).astype(int)
# Define a function to convert binary predictions to span segments
def binary_to_spans(binary_predictions):
    span_start_index = []
    span_end_index = []
    for prediction_row in binary_predictions:
        start = None
        start_inds = []
        end_inds = []
        
        for i, pred in enumerate(prediction_row):
            if pred == [1]:
                if start is None:
                    start = i
                    start_inds.append(i)
            else:
                if start is None:
                    continue
                else:
                    end_inds.append(i-1)
                    start = None
        if start is not None:
            end_inds.append(len(prediction_row) - 1)
        span_start_index.append(start_inds)
        span_end_index.append(end_inds)
    return span_start_index, span_end_index

# Convert binary predictions to span segments
span_start_index, span_end_index = binary_to_spans(binary_predictions)


In [252]:
# Find the indices of rows where task is equal to 2
task_2_indices = df_test[df_test['task'] == 2].index

# Update the span columns in df_test for rows where task is equal to 2
for i, idx in enumerate(task_2_indices):
    df_test.at[idx, 'span_start_index'] = span_start_index[i]
    df_test.at[idx, 'span_end_index'] = span_end_index[i]

In [253]:
df_test.head()

,Unnamed: 0,ID,text,claim,span_start_index,span_end_index,task
0,0,0,Of course we should have captured Osama Bin La...,1,[-2],[-2],1
1,1,1,covid19 will end soon amen covid19 will end so...,0,[-2],[-2],1
2,2,2,#Coronavirus #SanDiego #1 #Hotspot #92103 # Co...,1,[-2],[-2],1
3,3,3,@ICannot_Enough @elonmusk Yes it is. Because $...,0,[-2],[-2],1
4,4,4,Some people are saying black people are immune...,1,[-2],[-2],1


In [254]:
df_test.iloc[0]

Unnamed: 0                                                          0
ID                                                                  0
text                Of course we should have captured Osama Bin La...
claim                                                               1
span_start_index                                                 [-2]
span_end_index                                                   [-2]
task                                                                1
Name: 0, dtype: object

In [255]:
df_test.to_csv('submission2_1_1_3.csv', index=False)

In [231]:
lstm_model2.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ (None, 86, 128)        │     3,449,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ (None, 86, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 86, 1)          │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,497,221 (40.04 MB)

 Trainable params: 3,499,073 (13.35 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,998,148 (26.70 MB)